In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    beautifulsoup4-4.6.3       |           py35_0         139 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0        conda-forge
    ca-certificates: 2019.1.23-0          --> 2019.3.9-hecc5488_0 conda-forge
    certifi:         2018.8.24-py35_1     --> 2018.8.24-py35_1001 

In [2]:
!conda install -c conda-forge html5lib --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - html5lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    html5lib-1.0.1             |             py_0          89 KB  conda-forge

The following packages will be UPDATED:

    html5lib: 0.999999999-py35h0543385_0 --> 1.0.1-py_0 conda-forge


html5lib-1.0.1       | 89 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install -c conda-forge requests --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    requests-2.19.1            |           py35_1          94 KB  conda-forge

The following packages will be UPDATED:

    requests: 2.18.4-py35hb9e6ad1_1 --> 2.19.1-py35_1 conda-forge


requests-2.19.1      | 94 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [4]:
# Import libraries for web scraping
from bs4 import BeautifulSoup
import requests

In [11]:
# Download the HTML file for the page
!wget -q -O 'Postal_Codes.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### The following cells handle scraping the Wikipedia page to get the necessary data, then putting the data into a dataframe. For data scraping, I used the Requests library to get the webpage, then used BeautifulSoup to scrape the HTML data from the page.

In [50]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, "html5lib")

In [105]:
columns = soup.find("table").tr.text # Get table column names, these will be the column names in our dataframe
column_list = columns.split("\n") # Convert to list
for column in column_list: # loop through list and remove any empty strings
    if(column == ""):
        column_list.remove("")
print(column_list)

['Postcode', 'Borough', 'Neighbourhood']


In [217]:
# Get all the rows of data from the webpage
entry = []
row_list = []
for row in soup.find_all("table", class_ = "wikitable sortable"):
    if row is not None:
        entry.append(row.get_text())
    else:
        entry.append("Not assigned")
        
entry = [word for line in entry for word in line.split("\n\n\n")] # Split each row when 3 new line characters show up
entry = [word for line in entry for word in line.split("\n\n")] # This line handles the "\n\n" at the end of the list
for row in entry: # loop through list to remove empty strings left over by above lines
    if(row == ""):
        entry.remove("")
del entry[0] # Delete redundant row, we already saved it in columns

# Iterate through entry list, split each element at its newline character
# After this, each row of data will be contained in a list within a list, so it can be put into the dataframe
for element in entry:
    row_list.append(element.split("\n"))

print(row_list) # List that will populate the dataframe rows

[['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', "Queen's Park", 'Not assigned'], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'Cloverdale'], ['M9B', 'Etobicoke', 'Islington'], ['M9B', 'Etobicoke', 'Martin Gro

In [320]:
import pandas as pd
# Add rows to dataframe
df_postal = pd.DataFrame(row_list)
df_postal.columns = column_list
postal_code = {"Postcode" : "PostalCode"} # Rename Postcode column to PostalCode
df_postal.rename(postal_code, axis = 1, inplace = True)
df_postal.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Now that the data is properly in the dataframe, it needs to be cleaned as per the instructions.

In [321]:
# Remove all rows where Borough = Not assigned
df_postal = df_postal[df_postal.Borough != "Not assigned"]
df_postal.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [344]:
# Group together neighborhoods that are in the same postal code
df_grouped = df_postal.groupby(["PostalCode", "Borough"])["Neighbourhood"].apply(lambda x: "%s" % ', '.join(x)).to_frame()
df_grouped.reset_index(inplace = True)
df_grouped.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [345]:
# One entry has assigned borough but no assigned neighborhood, change "not assigned" to borough's name
df_grouped.loc[85, "Neighbourhood"] = "Queen's Park"
df_grouped

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### With dataframe cleaning done, print shape of the dataframe.

In [346]:
df_grouped.shape

(103, 3)